# Summary

This notebook merges the results for the left and right hemisphere, notebooks `1.2_MCB_with_bootstrap_left.ipynb` and `1.2_MCB_with_bootstrap_right.ipynb` respectively, and saves the MCBs made of statistically significant edges at $10\%$ level.

In [1]:
import sys
sys.path.insert(0, r"..\code")

import jax
import jax.numpy as jnp
import numpy as np
import pandas as pd

from utils import load_obj, save_obj
from copy import deepcopy

In [2]:
data_dir="../data/"
ts_dir="../data/TimeSeriesAAL/" 
processed="../data/processed/"
diffreg="../data/processed/diff_regions/"

In [3]:
replace=False
verbose=False

# Load results from left and right hemis

In [4]:
results_left=load_obj('1.2_bootstrap_ms_results_cut_idiosyncratic_left_0.2', processed)
results_right=load_obj('1.2_bootstrap_ms_results_cut_idiosyncratic_right_0.2', processed)

No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)


In [5]:
if replace:
    results = results_left.copy()
    for sample in list(results['multiscale'].keys()):
        results['multiscale'][sample]['right']=results_right['multiscale'][sample]['right'].copy()

# Assess statistical significance

In [6]:
J=5
hemis=['left','right']
nsamples=100
alpha=10
percentiles = jnp.array([alpha//2,50,100-alpha//2], dtype=jnp.int16)

if replace:
    results['bootstrap results']=dict()

    for hemi in hemis:
        results['bootstrap results'][hemi]=dict()
        for scale in range(J):
            results['bootstrap results'][hemi]['scale {}'.format(J-scale)]=dict()
            for sample in range(nsamples):
                itema = results['multiscale'][sample][hemi]['scale {}'.format(J-scale)]['Solution']
                itemc = results['multiscale'][sample][hemi]['scale {}'.format(J-scale)]['Causal tensor']
                if sample==0:
                    A_tilde=deepcopy(itema)
                    C_tilde=deepcopy(itemc)
                else:
                    A_tilde=jnp.concatenate((A_tilde,itema),axis=0)
                    C_tilde=jnp.concatenate((C_tilde,itemc),axis=0)

            C_tilde_l, C_median, C_tilde_u=jnp.percentile(C_tilde, percentiles, axis=0) #(K,K)
            C_bar = C_tilde_l*C_tilde_u
            A_b = jnp.where(C_bar>0,1,0)
            C_b = A_b*C_median

            results['bootstrap results'][hemi]['scale {}'.format(J-scale)]['Concatenated samples solutions']=A_tilde
            results['bootstrap results'][hemi]['scale {}'.format(J-scale)]['Concatenated samples causal tensors']=C_tilde
            results['bootstrap results'][hemi]['scale {}'.format(J-scale)]['Solution']=A_b
            results['bootstrap results'][hemi]['scale {}'.format(J-scale)]['Causal tensor']=C_b

    save_obj(results, '1.2_results_bootstrap_ms_cut_idiosyncratic_0.2', processed)

else:
    results=load_obj('1.2_results_bootstrap_ms_cut_idiosyncratic_0.2', processed)

# Export results

In [7]:
names = load_obj('region_names', processed)

In [8]:
groups=np.array([1,7,15,17,19,21,23,25,27,29,31,33,37,39,41,43,45,47,49,51,55,61,63,67,69,71,73, 87])
splitting_correct=(89*np.ones_like(groups)-groups)//2 #divide by 2 to account for left and right hemi
splitting_correct

array([44, 41, 37, 36, 35, 34, 33, 32, 31, 30, 29, 28, 26, 25, 24, 23, 22,
       21, 20, 19, 17, 14, 13, 11, 10,  9,  8,  1], dtype=int32)

In [9]:
if replace: writer = pd.ExcelWriter('1.2_Summary_statistics_bootstrap_results_multiscale.xlsx')

#preserved connections
lb=5
ub=95
hemis = ['right', 'left']

for scale in range(J):
    for hemi in hemis: 

        initial_ = results['multiscale'][0][hemi]['scale {}'.format(J-scale)]['Initial universe'] #this is equal for all bootstrap samples
        final_ = 2*results['bootstrap results'][hemi]['scale {}'.format(J-scale)]['Solution'] #this way, when I sum, I obtain -1 and +1
        weights_ = results['bootstrap results'][hemi]['scale {}'.format(J-scale)]['Concatenated samples causal tensors']
        inn=len(jnp.flatnonzero(initial_))
        fnn=len(jnp.flatnonzero(final_))

        print("Initial number of connections {}, final number of connections {}".format(inn, fnn))
        inout_=initial_+final_
        inout_nnz=inout_.nonzero()

        inout_df = pd.DataFrame(index=np.arange(inn), columns=['source', 'target', 'in/out', 'median', 'min', 'max', '{}%'.format(lb), '25%', '75%', '{}%'.format(ub)])

        for i in range(len(inout_df)):
            s=inout_nnz[0][i].item()
            t=inout_nnz[1][i].item()
            io=inout_[s,t].item()
            min_ = weights_[:,s,t].min().item()
            max_ = weights_[:,s,t].max().item()
            median_ = jnp.percentile(weights_[:,s,t], 50).item()
            liqr = jnp.percentile(weights_[:,s,t], 25).item()
            uiqr = jnp.percentile(weights_[:,s,t], 75).item()
            lbp = jnp.percentile(weights_[:,s,t], lb).item()
            ubp = jnp.percentile(weights_[:,s,t], ub).item()

            source_ = results['map_idx_to_regions'][s]['Region']
            target_ = results['map_idx_to_regions'][t]['Region']

            inout_df.iloc[i]=[source_,target_, io, median_, min_, max_, lbp, liqr, uiqr, ubp]

        if replace: inout_df.to_excel(writer, sheet_name='scale {} - {} hemi'.format(J-scale, hemi), index=True)

if replace:
    writer.save()
    writer.close()

Initial number of connections 88, final number of connections 9
Initial number of connections 96, final number of connections 10
Initial number of connections 96, final number of connections 15
Initial number of connections 89, final number of connections 17
Initial number of connections 73, final number of connections 22
Initial number of connections 76, final number of connections 26
Initial number of connections 70, final number of connections 25
Initial number of connections 59, final number of connections 26
Initial number of connections 46, final number of connections 14
Initial number of connections 46, final number of connections 18
